# Equivalente exacto a array_merge() de PHP
licencias_agentes = pd.concat([licencias_agentes_no_remunem, licencias_agentes_remunem], 
                             ignore_index=True)

In [3]:
# PASO 1: Setup e Imports
print("🚀 SICOSS Python")
print("=" * 60)

# Imports principales
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from SicossDataExtractor import *
from pandas.io.formats.excel import ExcelFormatter
from mapuche_licencias_extractor import *

# Configurar visualización
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

# Importar nuestro extractor
from SicossDataExtractor import *

print("✅ Imports completados")
print(f"🐼 Pandas: {pd.__version__}")
print(f"🔢 NumPy: {np.__version__}")
print(f"📊 Matplotlib configurado")
print(f"🔍 Seaborn: {sns.__version__}")

🚀 SICOSS Python
✅ Imports completados
🐼 Pandas: 2.3.0
🔢 NumPy: 2.3.1
📊 Matplotlib configurado
🔍 Seaborn: 0.13.2


## conexion a la base de datos

In [4]:
print("\n🔌 PASO 2: Estableciendo conexión a base de datos")
print("=" * 50)

def conectar_y_validar_bd():
    """Establece conexión y valida datos disponibles"""
    try:
        # Conectar
        db = DatabaseConnection('database.ini')
        print("✅ Conexión establecida exitosamente")
        
        # Validar datos básicos
        query_test = """
        SELECT 
            COUNT(*) as total_legajos
        FROM mapuche.dh01 
        WHERE tipo_estad = 'A'
        """
        
        result = db.execute_query(query_test)
        total_legajos = result.iloc[0]['total_legajos']
        
        print(f"📊 Total legajos activos: {total_legajos:,}")
        
        print("✅ Datos disponibles para procesamiento")
            
        return db
        
    except Exception as e:
        print(f"❌ Error conectando: {e}")
        print("💡 Verifica que database.ini esté configurado correctamente")
        return None


🔌 PASO 2: Estableciendo conexión a base de datos


In [5]:
# Ejecutar conexión
db_connection = conectar_y_validar_bd()

INFO:SicossDataExtractor:Ejecutando consulta: 
        SELECT 
            COUNT(*) as total_legajos
        FROM mapuche.dh01 
        WHERE tipo...


✅ Conexión establecida exitosamente
📊 Total legajos activos: 45,794
✅ Datos disponibles para procesamiento


## Configuracion de parametros

In [6]:
print("\n⚙️ PASO 3: Configurando parámetros SICOSS")
print("=" * 50)

# Configuración SICOSS (parámetros exactos)
config_sicoss = SicossConfig(
    tope_jubilatorio_patronal=4500000.00,
    tope_jubilatorio_personal=4500000.00,
    tope_otros_aportes_personales=99000000,
    trunca_tope=True,
    check_lic=False,
    check_retro=False,
    asignacion_familiar=False
)

# Parámetros de periodo (confirmados)
PERIODO_ANO = 2025
PERIODO_MES = 6
CODC_REPARTO = 'REPA'

# Límites de prueba progresivos
LIMITES_PRUEBA = {
    'mini': 10,      # Validación rápida
    'pequeño': 50,   # Prueba pequeña  
    'mediano': 200,  # Evaluación rendimiento
    'grande': 1000,  # Validación escalabilidad
    'xl': 5000       # Stress test (opcional)
}

print("✅ Configuración completada:")
print(f"   💰 Tope jubilatorio: ${config_sicoss.tope_jubilatorio_patronal:,.0f}")
print(f"   📅 Período: {PERIODO_ANO}/{PERIODO_MES}")
print(f"   🏢 Reparto: {CODC_REPARTO}")
print(f"   📊 Límites: {LIMITES_PRUEBA}")


⚙️ PASO 3: Configurando parámetros SICOSS
✅ Configuración completada:
   💰 Tope jubilatorio: $4,500,000
   📅 Período: 2025/6
   🏢 Reparto: REPA
   📊 Límites: {'mini': 10, 'pequeño': 50, 'mediano': 200, 'grande': 1000, 'xl': 5000}


# Extracción de datos

## Extracción de datos legajos

In [7]:
# TODO: probar con el extractor de licencias
# Instanciar el extractor de licencias
print("\n🏥 Configurando extractor de licencias...")

# El extractor de licencias utiliza la misma conexión de base de datos
extractor_licencias = MapucheLicenciasExtractor(db_connection)

print("✅ Extractor de licencias configurado correctamente")
print(f"   🔗 Conexión: {type(db_connection).__name__}")
print(f"   📋 Listo para extraer datos de licencias")

# Ejemplo de uso básico (comentado para no ejecutar automáticamente)
# df_licencias = extractor_licencias.extraer_licencias(
#     per_anoct=PERIODO_ANO,
#     per_mesct=PERIODO_MES,
#     legajo_especifico=None,  # None para todos los legajos
#     limite=LIMITES_PRUEBA['mini']  # Usar límite de prueba
# )



🏥 Configurando extractor de licencias...
✅ Extractor de licencias configurado correctamente
   🔗 Conexión: DatabaseConnection
   📋 Listo para extraer datos de licencias


## Extracción de datos conceptos

In [8]:
# TODO: probar con el extractor de licencias
config = MapucheConfig(db_connection)

In [9]:
config.get_fecha_inicio_periodo_corriente()

ERROR:mapuche_config:Error obteniendo fecha inicio período: psycopg2.connect() argument after ** must be a mapping, not DatabaseConnection


# EXTRACCION DE DATOS

## Obtener Legajos

In [10]:
extractor = SicossDataExtractor(db_connection)


df_legajos = extractor.extraer_legajos(
    per_anoct=2025,
    per_mesct=6,
    where_legajo="true"  # Para obtener todos los legajos
)


INFO:SicossDataExtractor:Ejecutando consulta: 
        SELECT
            DISTINCT(dh01.nro_legaj),
            (dh01.nro_cuil1::char(2)||LPAD(dh0...


In [11]:
print(f"Legajos obtenidos: {len(df_legajos)}")
df_legajos.head(2)

Legajos obtenidos: 112055


,nro_legaj,cuit,apyno,estado,conyugue,hijos,provincialocalidad,codigosituacion,codigocondicion,codigozona,codigoactividad,aporteadicional,trabajadorconvencionado,codigocontratacion,regimen,adherentes,licencia,importeimponible_9
0,9,2.004837e+10,GIBELLINI OSVALDO RENE,P,0,0,None,NaN,NaN,None,NaN,NaN,None,NaN,1,0.0,0,0
1,18,2.004137e+10,PETRUZZELLI LUIS SEBASTIA,P,0,0,None,NaN,NaN,None,NaN,NaN,None,NaN,1,0.0,0,0


In [12]:
df_legajos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112055 entries, 0 to 112054
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   nro_legaj                112055 non-null  int64  
 1   cuit                     112055 non-null  float64
 2   apyno                    112055 non-null  object 
 3   estado                   112055 non-null  object 
 4   conyugue                 112055 non-null  int64  
 5   hijos                    112055 non-null  int64  
 6   provincialocalidad       1496 non-null    object 
 7   codigosituacion          80061 non-null   float64
 8   codigocondicion          80061 non-null   float64
 9   codigozona               80061 non-null   object 
 10  codigoactividad          37726 non-null   float64
 11  aporteadicional          1496 non-null    float64
 12  trabajadorconvencionado  0 non-null       object 
 13  codigocontratacion       80061 non-null   float64
 14  regi

In [13]:
# Opción alternativa más específica para CUIT argentino
df_legajos['cuit'] = df_legajos['cuit'].apply(lambda x: f"{x:.0f}".zfill(11))

In [14]:
df_legajos.head(2)

,nro_legaj,cuit,apyno,estado,conyugue,hijos,provincialocalidad,codigosituacion,codigocondicion,codigozona,codigoactividad,aporteadicional,trabajadorconvencionado,codigocontratacion,regimen,adherentes,licencia,importeimponible_9
0,9,20048372571,GIBELLINI OSVALDO RENE,P,0,0,None,NaN,NaN,None,NaN,NaN,None,NaN,1,0.0,0,0
1,18,20041369486,PETRUZZELLI LUIS SEBASTIA,P,0,0,None,NaN,NaN,None,NaN,NaN,None,NaN,1,0.0,0,0


In [15]:
where_legajo_conceptos = extractor.construir_where_conceptos(df_legajos['nro_legaj'].tolist())

In [16]:
len(where_legajo_conceptos)

770091

## Extraer conceptos liquidados

In [17]:
df_conceptos = extractor.extraer_conceptos_liquidados(
    per_anoct=2025,
    per_mesct=6,
    where_legajo=where_legajo_conceptos
)

INFO:SicossDataExtractor:Ejecutando consulta: 
        WITH tipos_grupos_conceptos AS (
            SELECT
                dh16.codn_conce,
      ...


In [18]:
df_conceptos.shape

(1620241, 12)

In [21]:
df_conceptos.shape

(1596932, 12)

In [19]:
df_conceptos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620241 entries, 0 to 1620240
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   id_liquidacion   1620241 non-null  int64  
 1   impp_conce       1620241 non-null  float64
 2   ano_retro        1620241 non-null  int64  
 3   mes_retro        1620241 non-null  int64  
 4   nro_legaj        1620241 non-null  int64  
 5   codn_conce       1620241 non-null  int64  
 6   tipo_conce       1620241 non-null  object 
 7   nro_cargo        1620241 non-null  int64  
 8   nov1_conce       1620241 non-null  float64
 9   nro_orimp        1620241 non-null  int64  
 10  tipos_grupos     1620241 non-null  object 
 11  codigoescalafon  1620241 non-null  object 
dtypes: float64(2), int64(7), object(3)
memory usage: 148.3+ MB


In [20]:
df_conceptos.head()

,id_liquidacion,impp_conce,ano_retro,mes_retro,nro_legaj,codn_conce,tipo_conce,nro_cargo,nov1_conce,nro_orimp,tipos_grupos,codigoescalafon
0,38570000,185761.89,0,0,235459,101,C,376269,0.0,1,"[4, 29, 89]",DOCE
1,38570001,111457.13,0,0,235459,103,C,376269,12.0,2,"[4, 25, 89]",DOCE
2,38570002,37152.38,0,0,235459,137,C,376269,0.0,19,"[4, 41, 89]",DOCE
3,38570003,22.00,0,0,235459,174,S,376269,0.0,33,"[4, 30, 45, 89]",DOCE
4,38570004,11422.96,0,0,235459,192,C,376269,0.0,40,"[4, 30, 89]",DOCE


## Licencias

In [ ]:
# TODO: obtener licencias de los legajos como dataframe

In [ ]:
# crear instancia del extractor
licencias_extractor = MapucheLicenciasExtractor()

In [25]:
df_otra_actividad = extractor.extraer_otra_actividad(
    df_legajos['nro_legaj'].tolist()
)

INFO:SicossDataExtractor:Ejecutando consulta: 
        SELECT
			importe AS ImporteBrutoOtraActividad,
			importe_sac AS ImporteSACOtraActividad
	...


In [31]:
df_otra_actividad.head(2)

,importebrutootraactividad,importesacotraactividad


In [26]:
df_obra_social = extractor.extraer_codigos_obra_social(
    df_legajos['nro_legaj'].tolist()
)

INFO:SicossDataExtractor:Ejecutando consulta: 
        SELECT 
            nro_legaj,
            '000000' AS codigo_os
        FROM UNNEST(ARRAY[...


In [29]:
df_obra_social.tail(2)

,nro_legaj,codigo_os
110751,257495,000000
110752,257496,000000


In [30]:
df_obra_social.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110753 entries, 0 to 110752
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   nro_legaj  110753 non-null  int64 
 1   codigo_os  110753 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.7+ MB


In [32]:
# instanciar el procesador
procesador = SicossDataProcessor(config_sicoss)

In [33]:
datos_extraidos = {
    'legajos': df_legajos,
    'conceptos': df_conceptos,
    'otra_actividad': df_otra_actividad,
    'obra_social': df_obra_social
}

In [34]:
resultado = procesador.procesar_datos_extraidos(datos_extraidos)

INFO:SicossDataExtractor:Iniciando procesamiento de datos extraídos...


In [35]:
print(f"Legajos procesados: {len(resultado['legajos_procesados'])}")
print(f"Legajos válidos: {resultado['estadisticas']['legajos_validos']}")
print(f"Legajos rechazados: {resultado['estadisticas']['legajos_rechazados']}")

Legajos procesados: 35290
Legajos válidos: 35290
Legajos rechazados: 75463


In [36]:
type(resultado['totales'])

dict

In [37]:
print("\nTotales calculados:")
for concepto, valor in resultado['totales'].items():
    print(f"  {concepto}: ${valor:,.2f}")


Totales calculados:
  bruto: $59,457,459,660.36
  imponible_1: $96,132,608,849.21
  imponible_2: $59,457,459,660.36
  imponible_4: $96,132,608,849.21
  imponible_5: $96,132,608,849.21
  imponible_6: $0.00
  imponible_8: $96,132,608,849.21
  imponible_9: $0.00


In [38]:
df_sicoss_procesado = pd.DataFrame(resultado['legajos_procesados'])
df_sicoss_procesado.head(2)

,nro_legaj,cuit,apyno,estado,conyugue,hijos,provincialocalidad,codigosituacion,codigocondicion,codigozona,codigoactividad,aporteadicional,trabajadorconvencionado,codigocontratacion,regimen,adherentes,licencia,importeimponible_9,ImporteSAC,ImporteNoRemun,ImporteHorasExtras,ImporteZonaDesfavorable,ImporteVacaciones,ImportePremios,ImporteAdicionales,IncrementoSolidario,ImporteImponibleBecario,ImporteImponible_6,SACInvestigador,NoRemun4y8,ImporteTipo91,ImporteNoRemun96,ImporteBrutoOtraActividad,ImporteSACOtraActividad,codigo_os,Remuner78805,AsignacionesFliaresPagadas,ImporteImponiblePatronal,ImporteSACPatronal,ImporteImponibleSinSAC,IMPORTE_BRUTO,IMPORTE_IMPON,DiferenciaSACImponibleConTope,DiferenciaImponibleConTope,ImporteSACNoDocente,ImporteImponible_4,ImporteImponible_5,TipoDeOperacion,ImporteSueldoMasAdicionales
88,2510,27024444290,CASPARRI MARIA TERESA,J,0,0,None,1.0,14.0,1,37.0,NaN,None,8.0,1,0.0,0,0,9281943.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000000,9281943.07,0.0,2250000.00,2250000.00,0.0,2250000.00,9281943.07,7031943.07,0.0,9281943.07,9281943.07,9281943.07,1,0.0
150,4254,27023362649,SAUTU MARIA ANTONIA RUT,J,0,0,None,1.0,14.0,1,88.0,NaN,None,8.0,1,0.0,0,0,1566960.44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000000,1566960.44,0.0,1566960.44,1566960.44,0.0,1566960.44,1566960.44,0.00,0.0,1566960.44,1566960.44,1566960.44,1,0.0


In [41]:
filtro = df_sicoss_procesado['nro_legaj'] == 110830
df_sicoss_procesado[filtro]

,nro_legaj,cuit,apyno,estado,conyugue,hijos,provincialocalidad,codigosituacion,codigocondicion,codigozona,codigoactividad,aporteadicional,trabajadorconvencionado,codigocontratacion,regimen,adherentes,licencia,importeimponible_9,ImporteSAC,ImporteNoRemun,ImporteHorasExtras,ImporteZonaDesfavorable,ImporteVacaciones,ImportePremios,ImporteAdicionales,IncrementoSolidario,ImporteImponibleBecario,ImporteImponible_6,SACInvestigador,NoRemun4y8,ImporteTipo91,ImporteNoRemun96,ImporteBrutoOtraActividad,ImporteSACOtraActividad,codigo_os,Remuner78805,AsignacionesFliaresPagadas,ImporteImponiblePatronal,ImporteSACPatronal,ImporteImponibleSinSAC,IMPORTE_BRUTO,IMPORTE_IMPON,DiferenciaSACImponibleConTope,DiferenciaImponibleConTope,ImporteSACNoDocente,ImporteImponible_4,ImporteImponible_5,TipoDeOperacion,ImporteSueldoMasAdicionales
16976,110830,20183035860,CES ROMERO RAMON,A,0,0,None,1.0,1.0,1,17.0,NaN,None,8.0,1,0.0,0,0,8541877.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,000000,8541877.57,0.0,2250000.0,2250000.0,0.0,2250000.0,8541877.57,6291877.57,0.0,8541877.57,8541877.57,8541877.57,1,0.0


In [39]:
df_sicoss_procesado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35290 entries, 88 to 110734
Data columns (total 49 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   nro_legaj                      35290 non-null  int64  
 1   cuit                           35290 non-null  object 
 2   apyno                          35290 non-null  object 
 3   estado                         35290 non-null  object 
 4   conyugue                       35290 non-null  int64  
 5   hijos                          35290 non-null  int64  
 6   provincialocalidad             119 non-null    object 
 7   codigosituacion                35173 non-null  float64
 8   codigocondicion                35173 non-null  float64
 9   codigozona                     35173 non-null  object 
 10  codigoactividad                34709 non-null  float64
 11  aporteadicional                119 non-null    float64
 12  trabajadorconvencionado        0 non-null      ob

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd



# 1. Desanidar la columna 'tipos_grupos'
# Cada elemento de la lista en 'tipos_grupos' se convierte en una nueva fila.
df_grupos_exploded = df_conceptos.explode('tipos_grupos')

# 2. Contar la frecuencia de cada tipo de grupo
# cuántas veces aparece cada tipo de grupo en todas las liquidaciones.
frecuencia_grupos = df_grupos_exploded['tipos_grupos'].value_counts()

# 3. Preparar los datos para la gráfica (Top 15 para mayor claridad)
top_n = 15
grupos_top = frecuencia_grupos.head(top_n)

# 4. Generar la visualización
plt.style.use('seaborn-v0_8-darkgrid')
fig, ax = plt.subplots(figsize=(12, 8))

sns.barplot(x=grupos_top.values, y=grupos_top.index.astype(str), orient='h', palette='viridis', ax=ax)

# Añadir etiquetas y títulos
ax.set_title(f'Top {top_n} de Tipos de Grupos de Conceptos más Frecuentes', fontsize=16, weight='bold')
ax.set_xlabel('Frecuencia (Cantidad de Apariciones)', fontsize=12)
ax.set_ylabel('Código de Tipo de Grupo', fontsize=12)

# Añadir el valor exacto en cada barra para mayor claridad
for container in ax.containers:
    ax.bar_label(container, fmt='%d', padding=3, fontsize=10)

plt.tight_layout()
plt.show()

# 5. Mostrar la tabla de frecuencias
print(f"--- Frecuencia de los {top_n} Tipos de Grupos de Conceptos ---")
print(grupos_top)


In [ ]:
# Exportar el DataFrame de legajos procesados a CSV
df_legajos.to_csv('legajos_procesados.csv', 
                 index=False,
                 encoding='utf-8')

print("Archivo 'legajos_procesados.csv' exportado exitosamente")



In [ ]:
# Exportar el DataFrame de legajos procesados a Excel
# df_legajos.to_excel('legajos_procesados.xlsx', 
#                    index=False,
#                    sheet_name='Legajos',
#                    engine='openpyxl')

# print("Archivo 'legajos_procesados.xlsx' exportado exitosamente")


In [ ]:
filtro = resultado['legajos_procesados']['nro_legaj'] == 110830
resultado['legajos_procesados'][filtro]

## trabajar con el df de conceptos

In [25]:
nro_legajo_buscar = 110830
filtro_legajo = df_conceptos['nro_legaj'] == nro_legajo_buscar

df_filtrado = df_conceptos[filtro_legajo]

In [26]:
# Ver información del legajo filtrado
print(f"Legajo: {nro_legajo_buscar}")
print(f"Total conceptos: {len(df_filtrado)}")
print(f"Tipos de conceptos únicos: {df_filtrado['tipo_conce'].nunique()}")
print(f"Suma total importes: ${df_filtrado['impp_conce'].sum():,.2f}")

Legajo: 110830
Total conceptos: 54
Tipos de conceptos únicos: 5
Suma total importes: $8,541,877.57


In [28]:
df_conceptos_exploded = df_conceptos.explode('tipos_grupos')

In [45]:
tipo_grupo_buscar = 9
filtro = df_conceptos_exploded['tipos_grupos'] == tipo_grupo_buscar
df_conceptos_exploded[filtro].head(2)

,id_liquidacion,impp_conce,ano_retro,mes_retro,nro_legaj,codn_conce,tipo_conce,nro_cargo,nov1_conce,nro_orimp,tipos_grupos,codigoescalafon
1182680,40258424,86448.59,0,0,235459,123,C,376269,0.0,1,9,DOCE
1182689,40258445,1741772.77,0,0,119465,123,C,188952,0.0,1,9,NODO


In [48]:
# buscar legajo y tipo de grupo
tipo_grupo_buscar = 9
nro_legajo_buscar = 110830

filtro_tipo = df_conceptos_exploded['tipos_grupos'] == tipo_grupo_buscar
filtro_legajo = df_conceptos_exploded['nro_legaj'] == nro_legajo_buscar
filtro_combinado = filtro_tipo & filtro_legajo

In [51]:
print(f"Conceptos del legajo {nro_legajo_buscar} tipo {tipo_grupo_buscar} : {len(df_conceptos_exploded[filtro_combinado])}")

Conceptos del legajo 110830 tipo 9 : 1


In [52]:
df_conceptos_exploded[filtro_combinado]

,id_liquidacion,impp_conce,ano_retro,mes_retro,nro_legaj,codn_conce,tipo_conce,nro_cargo,nov1_conce,nro_orimp,tipos_grupos,codigoescalafon
1620032,41198521,2242844.8,0,0,110830,123,C,34135,0.0,1,9,NODO


In [61]:
## conceptos tipo C
tipo_conce_buscar = 'C'
filtro_tipo = df_conceptos['tipo_conce'] == tipo_conce_buscar
filtro_legajo = df_conceptos['nro_legaj'] == nro_legajo_buscar
filtro_combinado = filtro_tipo & filtro_legajo

df_conceptos[filtro_combinado]['impp_conce'].sum().__round__(2)

np.float64(5781151.14)